#  <span style="color:darkblue"> Ordinary Differential Equations - Initial Value Problems </span>

<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue"> Learning Objectives </span>

By the end of this module, students will be able to:

### Solve single first-order ODE initial value problems (IVPs)
- Solve a first-order ODE-IVP using the explicit Euler method
- Solve a first-order ODE-IVP using the implicit Euler method
- Solve a first-order ODE-IVP using the predictor-corrector method*
- Solve a first-order ODE-IVP by RK4
- Determine the truncation errors of different numerical integration methods*
- Determine the numerical stability of system of linear IVPs while using explicit numerical integration methods*

### Solve systems of first-order ODE-IVPs
- Solve a system of ODE-IVPs by explicit methods (explicit Euler, RK4)
- Solve a system of ODE-IVPs by implicit Euler
- Analyze the numerical stability of a system of ODE-IVPs*

### Solve single higher-order ODE IVPs
- Reformulate a single higher-order ODE-IVP into system of first-order ODE-IVPs*

\*Not covered in tutorial.

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Before starting this example, import the <b>Plots.jl</b> library by running the cell below.
</div>

In [ ]:
import Pkg; Pkd.add("Plots.jl"); using Plots


<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue"> First-order Ordinary Differential Equation, Initial Value Problem (ODE-IVP) </span>

Consider a first-order ODE:

\\[F\left(x,y,\frac{dy}{dx}\right)=0 \\]

with initial condition as $y(x=x_0)=y_0$. For most physical problems, the above ODE-IVP takes the form $F\left(x,y,\frac{dy}{dx}\right)=\frac{dy}{dx}-f(x,y)$ and can be rewritten as:

\\[\frac{dy}{dx}=f(x,y) \\]

The solution strategy for solving the above systems involves marching forward from the initial condition $y_0$ to generate approximations $y_1,y_2,\ldots,$ of the function $y(x)$ for specific values of the independent variable, $x_1,x_2,\ldots$.

# <span style="color:darkblue"> Explicit (Forward) Euler </span>

Explicit Euler is the simplest numerical approximation method for solving IVPs, in which we approximate the derivative with a forward finite difference. This iterative method (integration scheme) is given by:

\\[y_{i+1}:=y_i+hf(x_i,y_i)\\]

where $h$ is the iteration stepsize and denotes the increment between two successive values $h\equiv (x_{i+1}-x_i)$.

**Example 1:** Use explicit Euler to integrate 

\\[\frac{dy}{dx}=x^2y^{1/2} \\]

subject to the initial condition $y(0) = 1$. Plot the profile of $y(x)$ through $x \in [0,2]$.

*Solution:*

The right hand side function $f(x,y)=x^2y^{1/2}$ for implementing explicit Euler is defined in the function getf1() given below.



In [ ]:
# Define the explicit euler algoritm
function ivp_explicit_euler(f,x0,y0,h,n_out,i_out)
    xout = zeros(n_out + 1)             # Create storage for output
    yout = zeros(n_out + 1)             
    xout[1] = x0                        # Store initial values to first component of output
    yout[1] = y0
    x = x0                              # Initialize working x and y
    y = y0
    for j = 2:(n_out + 1)               # Outer loop over save points
        for i = 1:i_out                 # Advance i_out steps
            y += h*f(x,y)
            x += h
        end
        xout[j] = x                     # Save values after i_out steps
        yout[j] = y
    end
    return xout, yout
end

# Define righ-hand side function and initial condition
f_ex1(x,y) = x^2*y^(1/2)          
x0, y0 = 0.0, 1.0                                                # initial condition
h = 0.1                                                          # step size
n_out = 20                                                       # number of save points
i_out = 1                                                        # number of points between save
x_ex1,y_ex1 = ivp_explicit_euler(f_ex1, x0,y0,h,n_out,i_out)
        
# Plot y(x) through [0.0,2.0]
plot(x_ex1, y_ex1, 'r-o')
axis([0.0 2.0 1.0 5.0])                                          # Set axis limits
xlabels!('x')                                                    # Set x label to `x`
ylabels!('y')                                                    # Set y label to `y`

# <span style="color:darkblue"> Implicit (Backward) Euler </span>

Implicit Euler evaluates the derivative at the new values using a backward finite difference approximation. The scheme for implicit Euler is:

\\[y_{i+1}:=y_i+hf(x_{i+1},y_{i+1})\\]

As you can see, the only difference with explicit Euler is that the right-hand side function is evaluated at the next time $f(y_{i+1})$.  Most often, $y_{i+1}$ cannot be calculated analytically (explicitly). Rather, it needs to be computed from the solution of the algebraic equation

\\[R(y_{i+1})=y_{i+1}-y_i-hf(x_{i+1},y_{i+1})=0\\],

which, in general, will be nonlinear. The method of choice for its solution is Newton’s method, and it has to be applied in each step of the integration. Clearly, the implicit Euler method is more difficult to implement than the explicit Euler method, and the local truncation error is the same for both methods. The advantage of implicit Euler is that it is unconditionally stable (important for stability analysis).

**Example 2**: We now use implicit Euler to integrate the function in **Example 1**:

The implicit Euler formula would be written as the root of

\\[R(y_{i+1})=y_{i+1}-y_i-hx_{i+1}^2y_{i+1}^{1/2}=0\\]

To solve for the unknown variable $y_{i+1}$ using **Newton’s method**, we start with an initial guess $y_{i+1}^{(0)}$ and iterate using the formula

\\[y_{i+1}^{(k)}:=y_{i+1}^{(k)}-\frac{R(y_{i+1}^{(k)})}{R'(y_{i+1}^{(k)})}=y_{i+1}^{(k)}-\frac{y_{i+1}^{(k)} - y_i - h x_{i+1}^2 (y_{i+1}^{(k)})^{1/2}}{1-\frac{1}{2}h x_{i+1}^2 (y_{i+1}^{(k)})^{-1/2}}\\]

The functions for solving this problem using implicit Euler are defined as getf_IE and getdf_IE in the last section.

In [ ]:
function newton_solve_ie(f, df, y, yold, x, h, tol)
    f_val = f(y,yold,x,h);
    while abs(f) > tol
        y = y - f_val/df(y,x,h);
        f_val = f(y,yold,x,h);
    end
    return y
end

function ivp_implicit_euler(f,df,x0,y0,h,n_out,i_out,tol)
    xout = zeros(n_out + 1)                                   # Create storage for output
    yout = zeros(n_out + 1)             
    xout[1] = x0                                              # Store initial values to first component of output
    yout[1] = y0
    x = x0                                                    # Initialize working x and y
    y = y0
    for j = 2:n_out+1
        for i = 1:i_out                                       # Advance i_out steps
            y = newton_solve_ie(f, df, y, yold, x, h, tol)    # Solve nonlinear system to determine next step
            x = x + h;
        end
        xout[j] = x
        yout[j] = y
    end
    return xout, yout
end

f_ex2(y,yold,x,h) = f = y - yold - h*(x+h)^2*y^(0.5)
df_ex2(y,yold,x,h) = y - yold - h*(x+h)^2*y^(0.5

tol = 1E-8                                                                # nonlinear solve, absolute convergence tolerance
x_ex2,y_ex2 = ivp_implicit_euler(f_ex2,df_ex2,x0,y0,h,n_out,i_out,nl_tol) # run implicit Euler

ax = plot(x_ex2, y_ex2, 'b-s')                                            # Make plot for implicit
plot!(ax, x_ex1,y_ex1,'r-o')                                              # Make plot for explicit euler   
axis([0.0 2.0 1.0 6.5])                                                   # Set axis limits
xlabels!('x')                                                             # Set x label
ylabels!('y')                                                             # Set y label
legend('implicit Euler','explicit Euler','Location','northwest')          # Set legend

We observe a deviation of trajectories between using explicit Euler and implicit Euler. This is due to the accumulation of local truncation errors. 

# <span style="color:darkblue"> Solving Single First-order ODE-IVPs </span>

This method sort-of combines the explicit and implicit Euler methods, by using information on the slope both at $x_i$ and $x_{i+1}$. The algorithm consists of a predictor step which is identical to explicit Euler and an explicit corrector step, the scheme is given by:

\\[y_{i+1}^P=y_i+hf(x_i+y_i)\\
y_{i+1}^C=y_i+hf(x_{i+1},y_{i+1}^P)\\
y_{i+1}=\frac{y_{i+1}^P+y_{i+1}^C}{2}\\]

This method is classified as explicit.

# <span style="color:darkblue"> Runge-Kutta Methods  </span>
The predictor–corrector method and Euler’s method are special cases of a broader class of higher-order explicit methods known as Runge–Kutta methods. By far the most popular Runge–Kutta method is the particular fourth-order method:

\\[k_1:=f(x_i,y_i)\\
k_2:=f(x_i+\frac{h}{2},y_i+\frac{h}{2}k_1)\\
k_3:=f(x_i+\frac{h}{2},y_i+\frac{h}{2}k_2)\\
k_4:=f(x_i+h,y_i+h k_3)\\
y_{i+1}:=y_i+\frac{h}{6}(k_1+2k_2+2k_3+k_4)\\]

This particular method is very efficient and easy to program, and represents a good compromise between the number of functional evaluations and the global accuracy. Given its ubiquitousness, this particular fourth-order Runge–Kutta method is normally called **RK4**.

We consider using RK4 to integrate the function in **Example 1**:

In [ ]:
function ivp_RK4(f, x0,y0,h,n_out,i_out)
    xout = zeros(n_out + 1)                                   # Create storage for output
    yout = zeros(n_out + 1)             
    xout[1] = x0                                              # Store initial values to first component of output
    yout[1] = y0
    x = x0                                                    # Initialize working x and y
    y = y0
    for j = 2:n_out+1
        for i = 1:i_out
            k1 = f(x,y);
            k2 = f(x + 0.5*h,y + 0.5*h*k1);
            k3 = f(x + 0.5*h,y + 0.5*h*k2);
            k4 = f(x + h, y + h*k3);
            y = y + (h/6)*(k1 + 2*k2 + 2*k3 + k4);
            x = x + h;
        end
        xout(j) = x;
        yout(j) = y;
    end
    return xout, yout
end

[xout,y_RK4] = ivp_RK4(x0,y0,h,n_out,i_out); % RK4

% Plot y(x) through [0.0,2.0]
plot(tout,y_RK4,'k-v') % RK4
hold on
plot(x_IE,y_IE,'b-s') % implicit Euler
plot(x_EE,y_EE,'r-o') % explicit Euler
axis([0.0 2.0 1.0 6.5]); % Set axis limits
xlabel('x') % Set x label
ylabel('y') % Set y label
legend('RK4','implicit Euler','explicit Euler','Location','northwest') % Set legend
hold off 

Again, the truncation errors accumulate through integration resulting in the deviation of the  trajactories using different methods. Now, try to reduce the deviation by using smaller step size:
Interactive! Reduce the step size to h = 0.01 in the above code, you also need to modify the number of output points n_out for consistency.

<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue">  Solving Systems of Ordinary Differential Equations (IVPs)
    
In this section, we introduce numerical methods to solve a system of IVPs in a vector form as:

\\[\frac{d\mathbf{y}}{dx}=\mathbf{f}(\mathbf y) \\]

with the initial condition $\mathbf y (0)=\mathbf y_0$.  Here, we see that $\mathbf f:\mathbb R^n\to\mathbb R^n$ and $\mathbf y\in\mathbb R^n$, thus we have a system of  equations that must be solved simultaneously. 
    
    Explicit Methods
The methods that we have described for single equations can be readily adapted for systems of equations, and the accuracy properties of the methods are unchanged. In particular, the adaptation for explicit methods is really straightforward. All that changes is that you have to step through a system of equations instead of a single equation.
Explicit Euler
The explicit Euler method in this case has the form

Predictor-corrector
The predictor–corrector method is given by

RK4
The RK4 scheme becomes

    
Implicit Euler
The concepts for solving systems of ODEs are similar for implicit methods, but a bit more complicated. The form of the implicit Euler method becomes 
,
which is a system of nonlinear algebraic equations requiring Newton–Raphson for solving (at each step). We therefore form the residual vector
.
In the algorithm, we have an outer loop that is updating the values of  . At each step , we have an inner loop (with counter ) that involves first solving

and then updating the value for ,


Example 2: Use RK4 and implicit Euler to solve the system of equations

The initial conditions are  and . Plot the profiles of  and  through time horizon  using both methods.
Solution: 
We first consider to use RK4 for integration: we define the variable in the vector form , the right hand side function is then given by  and defined as getf_E2 which is written in the last section.

clear
t0 = 0.0; % initial condition
y0 = [1.0;2.0]; % initial condition
h = 1e-3; % step size
n_out = 50; % number of output points
i_out = 10; % frequency
[tout,y_RK4] = ivp_RK4_sys(t0,y0,h,n_out,i_out); % RK4
% getf_E2(y) is required

% Plot
plot(tout,y_RK4(:,1),'k-o')
hold on
plot(tout,y_RK4(:,2),'k-x')
axis([0.0 0.5 0.0 5.0])
xlabel('t')
ylabel('y')
legend('y_1','y_2','Location','northwest')
hold off

<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue">  Questions for reflection </span> 

- What are some reasons oscillations may be observed in the above profile? Are the oscillations expected or do they result from a numerical issue? What methods could you use to check this?
- How would the plot potentially differ if solved via the implict Euler method?